In [1]:
%matplotlib inline

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import numpy as np
import pandas as pd
import warnings

from matplotlib import pyplot as plt

from IPython.core.pylabtools import figsize
figsize(11, 9)

import collections

import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions

/home/aapopovkin/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/aapopovkin/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/aapopovkin/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/aapopovkin/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/aapopovkin/venv/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Handy snippet to reset the global graph and global session.
with warnings.catch_warnings():
  warnings.simplefilter('ignore')
  tf.reset_default_graph()
  try:
    sess.close()
  except:
    pass
  sess = tf.InteractiveSession()

This is model, where we should do binary classification - predict probability of A

   1) num_classes - number of classes
   
   2) each class has its popability of A
   
   3) prob - array of these probabilities; they are Q - coordinates for Hamiltonian MCMC
   
   4) prior (alpha, beta) are two float variables describing beta-distribution classes' popabilities.

In [3]:
def get_sample_data():
    ps_data_arr = np.array([
        [20, 10, 1, 0.499993], [20, 3, 2, 0.230211], [20, 8, 3, 0.236831], 
        [20, 7, 4, 0.246463], [20, 6, 5, 0.370862], [20, 5, 6, 0.320656], 
        [20, 10, 7, 0.519887], [20, 12, 8, 0.52845], [20, 8, 9, 0.453077], 
        [20, 8, 10, 0.431245], [20, 10, 11, 0.499243], [20, 9, 12, 0.471968], 
        [20, 2, 13, 0.152176], [20, 14, 14, 0.48496], [20, 6, 15, 0.246193]
    ])
    ps_data_pd=pd.DataFrame(data=ps_data_arr[0:, 0:],
             index=ps_data_arr[0:, 2],
             columns=["total_count", "clicks", "class_id", "true_p"],
             dtype=np.float32)
    ps_data_pd['class_id'] = ps_data_pd.class_id.astype('int32')
    
    return ps_data_pd

In [4]:
inv_alpha_transform = lambda y: np.log(y)  # Not using TF here.
fwd_alpha_transform = tf.exp

def _make_ps_prior(num_classes, dtype):
    raw_prior_alpha = tf.get_variable(
      name='raw_prior_alpha',
      initializer=np.array(inv_alpha_transform(5.), dtype=dtype))
    raw_prior_beta = tf.get_variable(
      name='raw_prior_beta',
      initializer=np.array(inv_alpha_transform(5.), dtype=dtype))   
    return tfd.Independent(
      tfd.Beta(
          fwd_alpha_transform(raw_prior_alpha) * tf.ones(num_classes),
          fwd_alpha_transform(raw_prior_beta) * tf.ones(num_classes)),
      reinterpreted_batch_ndims=1)

make_ps_prior = tf.make_template(name_='make_ps_prior', func_=_make_ps_prior)

def _make_ps_log_likelihood(prob, class_id, total_count):
    prob_c = tf.gather(prob, indices=tf.to_int32(class_id - 1), axis=-1)
    total_count_c = tf.gather(total_count, indices=tf.to_int32(class_id - 1), axis=-1)
    return tfp.distributions.Binomial(total_count=tf.to_float(total_count_c), probs=prob_c)

make_ps_log_likelihood = tf.make_template(name_='make_ps_log_likelihood', func_=_make_ps_log_likelihood)

def joint_log_prob(prob, total_count, clicks, class_id, dtype):
    num_classes = len(total_count)
    rv_prob = make_ps_prior(num_classes, dtype)
    rv_clicks = make_ps_log_likelihood(prob, class_id, total_count)
    return (rv_prob.log_prob(prob) + 
         tf.reduce_sum(rv_clicks.log_prob(clicks), axis=-1))


def approximate_alpha_and_beta(ps_data_pd):
    
    dtype = np.float32
    def unnormalized_posterior_log_prob(prob):
        return joint_log_prob(
            prob=prob,
            total_count=dtype(ps_data_pd.total_count.values),
            clicks=dtype(ps_data_pd.clicks.values),
            class_id=np.int32(ps_data_pd.class_id.values),
            dtype=dtype)

    step_size = tf.get_variable(
        'step_size',
        initializer=0.001,
        trainable=False)

    hmc = tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=unnormalized_posterior_log_prob,
        num_leapfrog_steps=1000,
        step_size=step_size,#0.01,
        step_size_update_fn=tfp.mcmc.make_simple_step_size_update_policy(target_rate=0.99),
        state_gradients_are_stopped=True)

    init_random_weights = tf.placeholder(dtype, shape=[len(ps_data_pd)])

    posterior_random_weights, kernel_results = tfp.mcmc.sample_chain(
        num_results=3,
        num_burnin_steps=0,
        num_steps_between_results=0,
        current_state=init_random_weights,
        kernel=hmc)

    loss = -tf.reduce_mean(kernel_results.accepted_results.target_log_prob)

    global_step = tf.train.get_or_create_global_step()

    learning_rate = tf.train.exponential_decay(
        learning_rate=0.1,
        global_step=global_step,
        decay_steps=2,
        decay_rate=0.998)

    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimizer.minimize(loss, global_step=global_step)

    init_op = tf.initialize_all_variables()
    with tf.variable_scope('make_ps_prior', reuse=True):
        prior_alpha = fwd_alpha_transform(tf.get_variable(
            name='raw_prior_alpha', dtype=dtype))
        prior_beta = fwd_alpha_transform(tf.get_variable(
            name='raw_prior_beta', dtype=dtype))

    init_op.run()
    w_ = 0.5 * np.ones([len(ps_data_pd)], dtype=dtype)
    
    
    maxiter = int(3000)
    num_accepted = 0
    num_drawn = 0
    for i in range(maxiter):
        [
          _,
          global_step_,
          loss_,
          posterior_random_weights_,
          kernel_results_,
          step_size_,
          prior_alpha_,
          prior_beta_
        ] = sess.run([
          train_op,
          global_step,
          loss,
          posterior_random_weights,
          kernel_results,
          step_size,
          prior_alpha,
          prior_beta
        ], feed_dict={init_random_weights: w_})
        
        w_ = posterior_random_weights_[-1, :]
        num_accepted += kernel_results_.is_accepted.sum()
        num_drawn += kernel_results_.is_accepted.size
        acceptance_rate = num_accepted / num_drawn
        if i % 1 == 0 or i == maxiter - 1:
            print('global_step:{:>4}  loss:{: 7.1f}  acceptance:{:.3f}  '
                  'step_size:{:.3f}  prior_alpha:{:.4f}  prior_beta:{:.4f}'.format(
                      global_step_, loss_.mean(),
                      acceptance_rate, step_size_,
                      prior_alpha_, prior_beta_)
            )

In [29]:
1 / (1 + np.exp(2.2))
196 / (832 + 196)

0.19066147859922178

In [ ]:
prior_alpha:55.1133  prior_beta:747.7076

In [5]:
dtype = np.float32

ps_data_pd = get_sample_data()

def unnormalized_posterior_log_prob(prob):
    return joint_log_prob(
        prob=prob,
        total_count=dtype(ps_data_pd.total_count.values),
        clicks=dtype(ps_data_pd.clicks.values),
        class_id=np.int32(ps_data_pd.class_id.values),
        dtype=dtype)

step_size = tf.get_variable(
    'step_size',
    initializer=0.001,
    trainable=False)

hmc = tfp.mcmc.HamiltonianMonteCarlo(
    target_log_prob_fn=unnormalized_posterior_log_prob,
    num_leapfrog_steps=1,
    step_size=step_size,#0.01,
    step_size_update_fn=tfp.mcmc.make_simple_step_size_update_policy(target_rate=0.99),
    state_gradients_are_stopped=True)

initial_state  = tf.placeholder(dtype, shape=[len(ps_data_pd)])

bootstrap_results = hmc.bootstrap_results(initial_state)

step = hmc.one_step(initial_state, bootstrap_results)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.


In [6]:
st = (np.random.normal(loc=0.5, scale=0.2, size=len(ps_data_pd)))
st[st < 0] = 0.001
st[st > 1] = 0.999

In [12]:
init_op = tf.global_variables_initializer().run()
sess.run(step_size.assign(0.34))
res = sess.run([bootstrap_results, step], feed_dict={initial_state: st})[1][1]
res#.accepted_results.grads_target_log_prob, res.proposed_results.grads_target_log_prob

MetropolisHastingsKernelResults(accepted_results=UncalibratedHamiltonianMonteCarloKernelResults(log_acceptance_correction=0.0, target_log_prob=-151.1382, grads_target_log_prob=[array([ 1.6157387e+01,  1.1412880e+02, -4.3469543e+01, -7.8273230e+00,
       -1.7525230e+01, -4.0278740e+01, -6.5233183e+00,  1.5987987e+04,
       -9.2642231e+00, -2.4163134e+01,  6.1668488e+01,  3.2638706e+01,
       -1.1561338e+01, -1.1413849e+01, -4.1079098e+01], dtype=float32)]), is_accepted=False, log_accept_ratio=-inf, proposed_state=array([ 1.5936775e+00,  6.5293989e+00, -1.9338468e+00,  2.7264917e-01,
       -8.5312968e-01, -1.5800996e+00,  4.7973242e-01,  9.2431696e+02,
       -1.4110649e-01, -9.7305721e-01,  3.2833693e+00,  2.5210164e+00,
       -5.6428713e-01, -3.5238785e-01, -1.7056563e+00], dtype=float32), proposed_results=UncalibratedHamiltonianMonteCarloKernelResults(log_acceptance_correction=-inf, target_log_prob=nan, grads_target_log_prob=[array([      nan,       nan,       nan, 16.972395,    

In [8]:
# 6.8840, 10.6375

# 7.2159, 10.9869

In [14]:
a, b, c = res.accepted_results.grads_target_log_prob[0], res.proposed_results.grads_target_log_prob[0], res.proposed_state

In [21]:
print (b[np.abs(a) < 15])
print (a[np.abs(a) < 15])
print (c[np.abs(a) < 15])


[16.972395  2.273705       nan       nan       nan]
[ -7.827323   -6.5233183  -9.264223  -11.561338  -11.413849 ]
[ 0.27264917  0.47973242 -0.14110649 -0.5642871  -0.35238785]
